In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データの傾向を分析するために、以下の観点から考察を行います。

### 1. カテゴリー別の売上状況
- **食品**: とても一般的な品目で、単価は200円から600円の間で、売上は安定している可能性があります。特に「りんご」や「牛乳」は売上数量が多く、需要が高そうです。
- **衣服**: Tシャツやジーンズは高単価ですが、売上数量が食品に比べて少ないかもしれません。特に、ジーンズは高価格帯のため、販売数が限られるかもしれません。
- **日用品**: 日用品のバスブラシは比較的安価ですが、よく与えられます。

### 2. 売上日の傾向
- 売上日のデータを確認することで、季節性や特定のイベント（セールなど）による影響を探ることができます。例えば、特定の休日やイベントに向けて、食品や衣服の売上が増えることがあるため、これらのパターンを特定することが重要です。

### 3. 利益分析
- 利益 = 売上 - 原価で計算できます。この計算を行い、各商品の利益状況を把握します。食品は原価が低めのため、比較的高い利益率を持つことが期待できますが、衣服は高単価ですが、原価も高いため利益率がどのように変わるかが重要です。

### 4. 売上数量と単価の相関
- 単価が高い商品は売上数量が低い場合が多く、逆に安い商品は売上数量が多いといった相関関係があるかもしれません。この相関を分析することで、どのような価格戦略が効果的かを見極めることができます。

### 5. 商品コードや商品名からのインサイト
- 特定の商品が他と比較して売上が著しく高い場合、その理由を分析します。商品名や特徴的な要素を掘り下げることで、マーケティング施策の参考になります。

### 結論
このデータセットを基に、定量的な分析（数値の可視化、集計）と定性的な分析（競合分析やマーケットトレンド）の両方を行うことで、より深いインサイトを得ることができます。さらに、機械学習などの手法を利用して、売上予測モデルを構築し、将来のトレンドを予測することも考えられます。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0                    売上データの傾向を分析するために、以下の観点から考察を行います。
1                                                    
2                                  ### 1. カテゴリー別の売上状況
3   - **食品**: とても一般的な品目で、単価は200円から600円の間で、売上は安定してい...
4   - **衣服**: Tシャツやジーンズは高単価ですが、売上数量が食品に比べて少ないかもしれま...
5             - **日用品**: 日用品のバスブラシは比較的安価ですが、よく与えられます。
6                                                    
7                                       ### 2. 売上日の傾向
8   - 売上日のデータを確認することで、季節性や特定のイベント（セールなど）による影響を探ること...
9                                                    
10                                        ### 3. 利益分析
11  - 利益 = 売上 - 原価で計算できます。この計算を行い、各商品の利益状況を把握します。食...
12                                                   
13                                  ### 4. 売上数量と単価の相関
14  - 単価が高い商品は売上数量が低い場合が多く、逆に安い商品は売上数量が多いといった相関関係が...
15                                                   
16                           ### 5. 商品コードや商品名からのインサイト
17  - 特定の商品が他と比較して売上が著しく高い場合

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
